In [ ]:
import numpy as np
import tensorflow as tf
import time

In [ ]:
# Global hyperparameters
batch_size = 20
max_grad_norm = 5
lr_decay = 0.5
learning_rate = 1.0
init_scale = 0.05

# LSTM hyperparameters
#word_emb_dim = 108     TBA after calculating word_vocab_size
num_steps = 35
hidden_size = 300
num_layers = 2
keep_prob = 0.5

In [ ]:
# possibles languages: ptb, fr, es, de, cs, ru ('ptb' means English PTB dataset)
lang = 'es'

pyphen_lang = {'ptb': 'en_US',
               'fr': 'fr_FR',
               'es': 'es',
               'de': 'de_DE',
               'cs': 'cs_CZ',
               'ru': 'ru_RU'}

if lang == 'ptb': 
  word_data = open('simple-examples/data/ptb.train.txt', 'r').read().replace('\n', '<eos>').split()
else:
  word_data = open('data/'+lang+'/train.txt', 'r').read().replace('\n', ' ').split()

words = list(set(word_data))
word_data_size, word_vocab_size = len(word_data), len(words)
print('data has %d words, %d unique' % (word_data_size, word_vocab_size))

word_to_ix = { word:i for i,word in enumerate(words) }
ix_to_word = { i:word for i,word in enumerate(words) }

def get_word_raw_data(input_file):
  if lang == 'ptb':
    data = open(input_file, 'r').read().replace('\n', '<eos>').split()
  else:
    data = open(input_file, 'r').read().replace('\n', ' ').split()
  return [word_to_ix[w] for w in data]

train_raw_data = get_word_raw_data('data/'+lang+'/train.txt')
valid_raw_data = get_word_raw_data('data/'+lang+'/valid.txt')
test_raw_data = get_word_raw_data('data/'+lang+'/test.txt')

def get_word_emb_dim(total):
  return round((total - 12 * hidden_size * hidden_size - hidden_size * (8 + word_vocab_size) - word_vocab_size) / \
         (word_vocab_size + 4 * hidden_size))

total = {'ptb': 5310115,
         'fr': 9820689,
         'es': 10430455,
         'de': 13317075,
         'cs': 16251327,
         'ru': 21353640}
word_emb_dim = get_word_emb_dim(total[lang])
print('word embedding size: ', word_emb_dim)

In [ ]:
class batch_producer(object):
  def __init__(self, raw_data, batch_size, num_steps):
    self.raw_data = raw_data
    self.batch_size = batch_size
    self.num_steps = num_steps
    
    self.batch_len = len(self.raw_data) // self.batch_size
    self.data = np.reshape(self.raw_data[0 : self.batch_size * self.batch_len],
                           (self.batch_size, self.batch_len))
    
    self.epoch_size = (self.batch_len - 1) // self.num_steps
    self.i = 0
  
  def __next__(self):
    if self.i < self.epoch_size:
      # batch_x and batch_y are of shape [batch_size, num_steps]
      batch_x = self.data[::, self.i * self.num_steps : (self.i + 1) * self.num_steps : ]
      batch_y = self.data[::, self.i * self.num_steps + 1 : (self.i + 1) * self.num_steps + 1 : ]
      self.i += 1
      return (batch_x, batch_y)
    else:
      raise StopIteration()

  def __iter__(self):
    return self

In [ ]:
class Model:
  
  word_embedding = tf.get_variable("word_embedding", [word_vocab_size, word_emb_dim], dtype=tf.float32,
                                  initializer=tf.random_uniform_initializer(-init_scale, init_scale))
  cell = tf.nn.rnn_cell.LSTMCell(hidden_size, forget_bias=0.0)
  
  def __init__(self, batch_size, need_reuse=False):
    self.x = tf.placeholder(tf.int32, [batch_size, num_steps])
    self.y = tf.placeholder(tf.int32, [batch_size, num_steps])
    
    words_embedded = tf.nn.embedding_lookup(self.word_embedding, self.x)
    lstm_input = tf.unpack(words_embedded, axis=1)

    self.apply_dropout()
    self.cell = tf.nn.rnn_cell.MultiRNNCell([self.cell] * num_layers)
    
    self.init_state = self.cell.zero_state(batch_size, dtype=tf.float32)
    with tf.variable_scope('lstm_rnn', reuse=need_reuse):
      outputs, self.state = tf.nn.rnn(self.cell, lstm_input, dtype=tf.float32, initial_state=self.init_state)

    with tf.variable_scope('softmax_params', reuse=need_reuse):
      weights = tf.get_variable('weights', [hidden_size, word_vocab_size], 
                                initializer=tf.random_uniform_initializer(-init_scale, init_scale), 
                                dtype=tf.float32)
      biases = tf.get_variable('biases', [word_vocab_size], 
                               initializer=tf.random_uniform_initializer(-init_scale, init_scale),
                               dtype=tf.float32)

    output = tf.reshape(tf.concat(1, outputs), [-1, hidden_size])
    logits = tf.matmul(output, weights) + biases
    loss = tf.nn.seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(self.y, [-1])],
            [tf.ones([batch_size * num_steps], dtype=tf.float32)])
    self.cost = tf.reduce_sum(loss) / batch_size

 
  def apply_dropout(self):
    self.cell = tf.nn.rnn_cell.DropoutWrapper(self.cell, output_keep_prob=keep_prob)

In [ ]:
class Train(Model):
  
  def __init__(self, batch_size):
  
    super(Train, self).__init__(batch_size)
    
    self.lr = tf.Variable(0.0, trainable=False, dtype=tf.float32)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars), max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self.train_op = optimizer.apply_gradients(zip(grads, tvars),
                                              global_step=tf.contrib.framework.get_or_create_global_step())
    
    self.new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
    self.lr_update = tf.assign(self.lr, self.new_lr)

  def assign_lr(self, session, lr_value):
    session.run(self.lr_update, feed_dict={self.new_lr: lr_value})

In [ ]:
class Eval(Model):
  
  def apply_dropout(self):
    pass

In [ ]:
train = Train(batch_size)
valid = Eval(batch_size, need_reuse=True)
test = Eval(1, need_reuse=True)

In [ ]:
def model_size():
  params = tf.trainable_variables()
  size = 0
  for x in params:
    sz = 1
    for dim in x.get_shape():
      sz *= dim.value
    size += sz
  return size

print('Model size is: ', model_size())

def my_model_size():
  return word_emb_dim * word_vocab_size + \
         4 * (hidden_size * word_emb_dim + hidden_size * hidden_size + hidden_size) +\
         4 * (hidden_size * hidden_size + hidden_size * hidden_size + hidden_size) +\
         hidden_size * word_vocab_size + word_vocab_size

print('My model size: ', my_model_size())

In [ ]:
num_epochs = 25
display_freq = 200

init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  prev_perplexity = float('inf')
  
  for epoch in range(num_epochs):
    start_time = time.time()
    train.assign_lr(sess, learning_rate)
    
    iters = 0
    costs = 0
    
    train_batches = batch_producer(train_raw_data, batch_size, num_steps)
        
    training_state = None

    for batch in train_batches:
      if not training_state: training_state = sess.run(train.init_state)
      _, c, training_state, my_lr = sess.run([train.train_op, train.cost, train.state, train.lr],
                                             feed_dict={train.x: batch[0], train.y: batch[1], 
                                                        train.init_state: training_state})
      
      costs += c
      if iters % (display_freq*num_steps) == 0 and iters != 0:
        print('step =', iters/num_steps, end=', ')
        print('perplexity =', np.exp(costs / iters), end=', ')
        print('learning rate =', my_lr, end=', ')
        print('speed =', round(iters * batch_size / (time.time() - start_time)), ' wps')

      iters += num_steps
    
    print('epoch ', epoch+1, end = ': ')
    print('perplexity =', np.exp(costs / iters), end=', ')
    print('learning rate =', my_lr)
    
    
    # Get validation set perplexity
    valid_costs = 0
    valid_state = None
    valid_iters = 0

    valid_batches = batch_producer(valid_raw_data, batch_size, num_steps)
    
    for valid_batch in valid_batches:
      
      if not valid_state: valid_state = sess.run(valid.init_state)
      c, valid_state = sess.run([valid.cost, valid.state], 
                                feed_dict={valid.x: valid_batch[0], valid.y: valid_batch[1], 
                                           valid.init_state: valid_state})

      valid_costs += c
      valid_iters += num_steps
    
    cur_perplexity = np.exp(valid_costs / valid_iters)
    print('Validation set perplexity =', cur_perplexity)

    if prev_perplexity - cur_perplexity < 1:
      learning_rate *= lr_decay
    prev_perplexity = cur_perplexity

    
    # Get test set perplexity
    test_costs = 0
    test_state = None
    test_iters = 0

    test_batches = batch_producer(test_raw_data, 1, num_steps)
    
    for test_batch in test_batches:

      if not test_state: test_state = sess.run(test.init_state)
      c, test_state = sess.run([test.cost, test.state], 
                                feed_dict={test.x: test_batch[0], test.y: test_batch[1], 
                                           test.init_state: test_state})

      test_costs += c
      test_iters += num_steps
    
    print('Test set perplexity =', np.exp(test_costs / test_iters))
       

    print('-' * 100)